In [1]:
# IMPORTS
from ipywidgets import HBox, VBox, Label, Layout, AppLayout, Output, Tab, Text
from IPython.display import display
import sys, os
import traceback
from pathlib import Path
from pprint import pprint
import pandas as pd

# append parent of cwd to sys path to find modules
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# surpress pygame version prompt
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

# enable matplotlib widgets
%matplotlib widget

from definitions import ROOT_DIR
from src.app_explore import AppExplore as App
from src.io.plot import multi_evaluation_bars, two_multibar_plots, animated_plotly_pianoroll, multitrack_plotly_pianoroll
from src.io.output import play_button

# widget objects
from dashboards.widgets import paginationSlider, iconButton, exploreHeading, databaseIdTextfield, showButton

# widget functions
from dashboards.widgets import h2_heading

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
##########################
#C#    OUTPUT & APP    ###
##########################

log = Output()

summary_container = Output()
pianoroll_container = Output()
input_play_button_container = Output()
gen_play_button_container = Output()
output_play_button_container = Output()
evaluation_container = Output()
meta_container = Output()
generation_info = Output()

# init
app = App(log=log)

In [3]:
##########################
#M#   SELECTION DATA   ###
##########################


# UI Elements
generation_page_slider = paginationSlider(1, '')
refreshButton = iconButton('refresh', 'Load sample')
leftButton = iconButton('arrow-left', 'Previous')
rightButton = iconButton('arrow-right', 'Next')

In [4]:
##########################
#C#     CONTROLLER     ###
##########################

input_cache = str(ROOT_DIR / Path('midi\\tmp\\mgeval_cache.mid'))

result = None

def run_explore(b):
    try:
        # reset output containers
        summary_container.clear_output()            

        # run
        result = app.run(databaseIdTextfield.value)
        generation_page_slider.value = len(result['generations']) - 1


        display_details(None)
    except:
        # printing stack trace
        with log:
            traceback.print_exc()


def display_details(b):
    try:
        # reset output containers
        pianoroll_container.clear_output()
        input_play_button_container.clear_output()
        gen_play_button_container.clear_output()
        output_play_button_container.clear_output()
        evaluation_container.clear_output()
        meta_container.clear_output()

        if app.result is None:
            with pianoroll_container:
                print('ERROR: No results available.')
                return
        
        if b is not None and b.tooltip is "Previous":
            x = generation_page_slider.value - 1
            if x < 0: return
        elif b is not None and b.tooltip is "Next":
            x = generation_page_slider.value + 1
            if x >= len(app.result['generations']): return
        else:
            x = generation_page_slider.value

        cr_set = app.result['generations'][x]
        generation_page_slider.value = x

        # create pianoroll plots from results and print them
        midi_list = [cr_set.input_sequence.sequence, cr_set.generated_base_sequence.sequence, cr_set.output_sequence.sequence]
        name_list = ['Input', 'Generation', 'Result']

        multitrack_plotly_pianoroll(midi_list, name_list, 'Sequences', pianoroll_container)


        # display play buttons for audio
        with input_play_button_container: 
            display(h2_heading('Input:'))
        play_button(midi_list[0], input_play_button_container)
        with gen_play_button_container: 
            display(h2_heading('Generation:'))    
        play_button(midi_list[1], gen_play_button_container)
        with output_play_button_container: 
            display(h2_heading('Adaptation:'))
        play_button(midi_list[2], output_play_button_container)


        # create evaluation plots
        eval_data = [cr_set.generation_similarity['normalized'], cr_set.output_similarity['normalized']]
        multi_evaluation_bars(eval_data, evaluation_container, ['Generated Sequence', 'Adapted Sequence'])


        # print avg sim/var
        numeric_data = pd.DataFrame()
        numeric_data = numeric_data.append(pd.Series(app.evaluation.calc_meta_scores(eval_data[0]), name='Generation Similarity:'))
        numeric_data = numeric_data.append(pd.Series(app.evaluation.calc_meta_scores(eval_data[1]), name='Adaptation Similarity:'))

        with meta_container:
            display(numeric_data)

        # print meta data
        with meta_container:
            display(h2_heading('Input Analysis:'))
            pprint(cr_set.input_sequence.analysis)
            display(h2_heading('Generation Analysis:'))
            pprint(cr_set.output_sequence.analysis)
            display(h2_heading('Process Meta Data:'))
            pprint(cr_set.output_sequence.meta)

            if app.result['db_generation_ids'] is not None:
                print('Database ID of Gen Base (Table MIDIs): ' + str(app.result['db_generation_ids'][x]))
    except:
        # printing stack trace
        with log:
            traceback.print_exc()

def reload_details(change):
    display_details(None)


# settings button click handlers
showButton.on_click(run_explore)

# pagination event handlers
refreshButton.on_click(display_details)
leftButton.on_click(display_details)
rightButton.on_click(display_details)
generation_page_slider.observe(reload_details, names='value')

In [5]:
##########################
#V#  SETTINGS SIDEBAR  ###
##########################

side_bar_layout = Layout(border='1px solid grey', padding='10px')

settings_box = VBox([
        exploreHeading,
         HBox([Label('Generation ID:'), databaseIdTextfield]),
        showButton,
    ], layout = side_bar_layout)

In [6]:
##########################
#V#    RESULTS AREA    ###
##########################

main_layout = Layout(padding='10px')

# AVG & SUMMARY
summary_box = summary_container

# DETAILS
pagination_box = HBox([leftButton, rightButton, Label('Sample Nr.:'), generation_page_slider, refreshButton])
play_buttons = HBox([input_play_button_container, gen_play_button_container, output_play_button_container])
details_box = VBox([pagination_box, pianoroll_container, play_buttons, evaluation_container, meta_container], layout=main_layout)

In [7]:
##########################
#V#        TABS        ###
##########################

tabs = Tab()
tabs.children = [summary_box, details_box]
tabs.set_title(0, "Summary")
tabs.set_title(1, "Detailed Results")

In [8]:
##########################
#V#     APP LAYOUT     ###
##########################

AppLayout(left_sidebar=settings_box,
          center=tabs,
          header=None,
          footer=log,
          right_sidebar=None,
          pane_widths=[1,2,0],
          pane_heights=[0,4,'200px'])

AppLayout(children=(Output(layout=Layout(grid_area='footer')), VBox(children=(HTML(value='<h2>Select an entry …

In [9]:
# display_details(None)

In [10]:
# run_explore(showButton)